In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import GRU, Dense
import datetime
from IPython.display import display
import tensorflow as tf
import os

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load the data
file_path = 'Tether_to_IDR_Conversion.xlsx'
data = pd.read_excel(file_path)

# Convert date to datetime format
data['Tanggal'] = pd.to_datetime(data['Tanggal'], format='%d/%m/%Y')
data.set_index('Tanggal', inplace=True)

# Ensure the data is sorted by date
data = data.sort_index()

# Define short-term and long-term windows for DMA
short_window = 5
long_window = 20

# Calculate short-term and long-term moving averages
data['SMA'] = data['Terakhir_IDR'].rolling(window=short_window, min_periods=1).mean()
data['LMA'] = data['Terakhir_IDR'].rolling(window=long_window, min_periods=1).mean()

# Display the first few rows of the data
display(data)

In [ ]:
# Prediction with Double Moving Average (DMA)
data['DMA_Prediction'] = data['SMA']

# Extend DMA predictions for future dates dynamically
last_values = data['SMA'].dropna().values[-short_window:]
future_dates = [data.index[-1] + datetime.timedelta(days=i) for i in range(1, 6)]
future_sma = []

for i in range(5):
    new_sma = np.mean(np.append(last_values, [last_values[-1]])) # Simulate next SMA value
    future_sma.append(new_sma)
    last_values = np.append(last_values[1:], new_sma) # Shift the window

future_df_dma = pd.DataFrame({'DMA_Prediction': future_sma}, index=future_dates)

# Combine the original and future DMA predictions
dma_data = pd.concat([data, future_df_dma])

# Calculate differences and percentage differences for DMA
dma_data['Difference'] = dma_data['Terakhir_IDR'] - dma_data['DMA_Prediction']
dma_data['Percentage_Difference'] = dma_data['Difference'] / dma_data['Terakhir_IDR'] * 100

# Display the DMA predictions
display(dma_data.tail(10))

# Plot DMA predictions
plt.figure(figsize=(14, 7))
plt.plot(dma_data.index, dma_data['Terakhir_IDR'], label='Harga Asli', linewidth=2.5)
plt.plot(dma_data.index, dma_data['DMA_Prediction'], label='Prediksi DMA', color='orange')
plt.axvspan(future_dates[0], future_dates[-1], color='orange', alpha=0.3, label='Area Prediksi')
plt.xlabel('Tanggal')
plt.ylabel('Harga (IDR)')
plt.title('Prediksi DMA USDT dengan Harga Asli')
plt.legend()
plt.grid(True)
plt.show()

# Display comparison table for DMA
dma_comparison = dma_data[['Terakhir_IDR', 'DMA_Prediction', 'Difference', 'Percentage_Difference']]
display(dma_comparison.tail(10))

In [ ]:
# Prepare data for GRU
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Terakhir_IDR']].fillna(method='ffill'))

# Create a function to prepare the dataset for GRU
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 20
X, Y = create_dataset(scaled_data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split into train and test sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

# Adjust the GRU model
model_path = 'GRU_model.h5'
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=(look_back, 1), recurrent_activation='sigmoid'))
    model.add(GRU(50, recurrent_activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, Y_train, epochs=20, batch_size=1, verbose=2)
    model.save(model_path)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

# Prepare data for future predictions
future_days = 5
future_predictions = []

last_sequence = scaled_data[-look_back:]
for _ in range(future_days):
    prediction = model.predict(np.reshape(last_sequence, (1, look_back, 1)))
    future_predictions.append(prediction[0, 0])
    last_sequence = np.append(last_sequence[1:], prediction, axis=0)

future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Append future predictions to the data
future_df_gru = pd.DataFrame(future_predictions, index=future_dates, columns=['GRU_Prediction'])

# Combine the original and future data
gru_data = data.copy()
gru_data['GRU_Prediction'] = np.nan
gru_data = pd.concat([gru_data, future_df_gru])

# Calculate differences and percentage differences for GRU
gru_data['Difference'] = gru_data['Terakhir_IDR'] - gru_data['GRU_Prediction']
gru_data['Percentage_Difference'] = gru_data['Difference'] / gru_data['Terakhir_IDR'] * 100

# Display the GRU predictions
display(gru_data.tail(10))

# Plot GRU predictions along with actual prices
plt.figure(figsize=(14, 7))
plt.plot(gru_data.index, gru_data['Terakhir_IDR'], label='Harga Asli', linewidth=2.5)
plt.plot(gru_data.index, gru_data['GRU_Prediction'], label='Prediksi GRU', color='green')
plt.axvspan(future_dates[0], future_dates[-1], color='green', alpha=0.3, label='Area Prediksi')
plt.xlabel('Tanggal')
plt.ylabel('Harga (IDR)')
plt.title('Prediksi GRU USDT dengan Harga Asli')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display comparison table for GRU
gru_comparison = gru_data[['Terakhir_IDR', 'GRU_Prediction', 'Difference', 'Percentage_Difference']]
display(gru_comparison.tail(10))

# Combine DMA and GRU predictions
combined_data = dma_data.copy()
combined_data['GRU_Prediction'] = gru_data['GRU_Prediction']

# Display the combined predictions
display(combined_data.tail(10))

# Plot combined predictions
plt.figure(figsize=(14, 7))
plt.plot(combined_data.index, combined_data['Terakhir_IDR'], label='Harga Asli', linewidth=2.5)
plt.plot(combined_data.index, combined_data['DMA_Prediction'], label='Prediksi DMA', color='orange')
plt.plot(combined_data.index, combined_data['GRU_Prediction'], label='Prediksi GRU', color='green')
plt.axvspan(future_dates[0], future_dates[-1], color='gray', alpha=0.3, label='Area Prediksi')
plt.xlabel('Tanggal')
plt.ylabel('Harga (IDR)')
plt.title('Perbandingan Prediksi USDT')
plt.legend()
plt.grid(True)
plt.show()

# Save combined predictions to Excel
combined_data.to_excel('Tether_Combined_Predictions.xlsx')